### Get the User's Desired LLM

In [1]:
import langchain_test as lt 
import methods

model = lt.get_user_choice()
print("You have chosen", model.name)

Please input your choice of LLM
You have chosen ChatGPT


### Get User's Calendar

In [2]:
calendar = methods.get_calendar()
user_query = lt.get_user_query()

What change would you like to make to your calendar


### Set Up Prompts 

In [17]:
from langchain_core.prompts import ChatPromptTemplate

intent_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "The user's calendar is {user_calendar}"),
    ("system", "Here is the current time - {time}"),
    ("human", "Identify the type of query that the following query is: {user_query}. Your response should be a single word that describes the intent of the query, and should be one of the following: createEvent, moveEvent, deleteEvent, checkSchedule, listEvents, updateEvent."),
])

update_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "append _format to the following user intent - {user_intent}"),
    ("system", "Only output the appended word. Your answer should be one word in the following format: userintent_format")
])

update_json_template = ChatPromptTemplate.from_messages([
    ("system", "Use the current time and user's calendar to update the following format - {format}"),
    ("human", "What was the original query")
])

### Build The Chain

In [18]:
from langchain_core.runnables import RunnablePassthrough
import formats

lcel_chain = intent_prompt_template | model.llm | {'user_intent':RunnablePassthrough()} |update_prompt_template | model.llm | {'format':RunnablePassthrough()} | update_json_template | model.llm

out = lcel_chain.invoke({"user_calendar":calendar, "time":methods.current_time, "user_query":user_query})
print("LCEL:", out.content)

LCEL: updateEvent_format
